## Etape 1 : Scraping des résultats depuis bases.athle

Objectifs :
+ créer un pipeline pour récupérer les données d'une compétition d'athlétisme (course sur route, trail, cross, ...)
+ retourner ces données sous la forme d'une table relationnelle
+ par la suite, exploiter cette table pour créer des statistiques et visualisations sur les compétitions

Format d'une URL de bases.athle :

+ ? avant la requête
+ & pour ajouter des paramètres à la recherche 

bases.athle.fr/asp.net/liste.aspx**?frmbase=resultats**&**frmmode=1**&**frmespace=0**&**frmcompetition=**274211

Liste des paramètres pour une recherche par athlète :
+ frmsaison
+ frmclub
+ frmnom
+ frmprenom
+ frmsexe
+ frmlicence
+ frmdepartement
+ frmligue
+ frmcomprch


### Installation de la libraire lxml

In [1]:
!pip install lxml

### Import des librairies requises

In [2]:
import lxml
import urllib
import bs4
import pandas as pd
import re
from urllib import request

In [13]:
#Exemple de résultats d'une compétition
url="https://bases.athle.fr/asp.net/liste.aspx?frmbase=resultats&frmmode=1&frmespace=0&frmcompetition=253657"

In [14]:
request_text = request.urlopen(url).read()

In [40]:
page = bs4.BeautifulSoup(request_text, "lxml")

## Date, nom, lieu, ligue, label et département de la compétition

In [41]:
header = page.find('div', {'class' : "mainheaders"})
header=str(header)
print(header)

<div class="mainheaders">06/11/22 - 10 km de Riorges<br/><span style="font-size:13px">Riorges - ARA - 042 - Label Régional</span></div>


#### Nom de la compétition

In [42]:
re_nom = re.compile("(?<=\- )(.*?)(?=\<)")
text = re_nom.findall(header)
nom = text[0]
sous_titre = text[1]
print(nom)

10 km de Riorges


#### Lieu de la compétition

In [43]:
re_lieu = re.compile("(?<=\>)(\D*?)(?=\ -)")
lieu = re_lieu.findall(header)[0]
print(lieu)

Riorges


#### Date de la compétition

In [44]:
re_date = re.compile("[0-9]{2}/[0-9]{2}/[0-9]{2}")
date = re_date.findall(header)[0]
print(date)

06/11/22


#### Ligue de la compétition

In [47]:
soustitre = nom[1]
re_ligue = re.compile("[A-Z]{3}")
ligue = re_ligue.findall(sous_titre)[0]
print(ligue)

ARA


#### Département de la compétition

In [49]:
re_dept = re.compile("[0-9]{3}")
dept = re_dept.findall(sous_titre)[0]
print(dept)

042


#### Label de la compétition (en cours)

In [50]:
re_label = re.compile("(?<=Label ).*(?<!')")
label = re_label.findall(sous_titre)[0]
print(label)

Régional


#### Données sous forme de dataframe

In [60]:
compet= {'nom': nom,
      'lieu': lieu,
      'date': date,
      'ligue': ligue,
      'dept': dept,
      'label': label}

In [65]:
compet['dept']

'042'

In [62]:
print(page.find("title"))

<title>les Résultats des Compétitions</title>


In [70]:
print(page.find("mainheaders"))

None


In [67]:
print("Il y a", len(page.findAll("tr")), "éléments dans la page qui sont des <tr>")

Il y a 257 éléments dans la page qui sont des <tr>


In [68]:
print(page.find('a', {'title' : 'Records principaux pour cette épreuve'}))

<a class="white" href="/asp.net/liste.aspx?frmbase=records&amp;frmmode=2&amp;frmespace=0&amp;frmcompetition=253657&amp;frmepreuve=261&amp;frmsexe=M&amp;frmserie=10 Km Route TC" title="Records principaux pour cette épreuve">10 Km Route TC</a>


In [69]:
print(page.find('tr'))

<tr> <td style="width:2px"><div class="separator"></div></td> <td class="barButtons"><span class="barButtonsOff" onclick="history.back()" onmouseout="this.className='barButtonsOff'" onmouseover="this.className='barButtonsOn'" style="width:25px;background-image:url('/images/v3/bar.back.gif')" title="Retour"> </span></td> <td style="width:2px"><div class="separator"></div></td><td class="barInputs"> <select class="barSelect" onchange="document.location='/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;frmposition=' + this.options[this.selectedIndex].value"><option selected="selected" value="0">Page &gt; 001/002 &lt;</option><option value="1">Page &gt; 002/002 &lt;</option></select>  <select class="barSelect" onchange="document.location='/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;frmtri=' + this.options[this.selectedIndex].value"><option selected="selected" value="0">Tri &gt; Cat./Sex./

In [71]:
print('EOF')

EOF
